In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ast 

# import os
# if os.name == 'posix' :
#     plt.rc('font', family='AppleGothic')
# else :
#     plt.rc('font', family='Malgun Gothic')
    
plt.rc('axes', unicode_minus=False)

plt.rcParams['font.family'] = 'NanumGothic' 
plt.rcParams['axes.unicode_minus'] = False
import koreanize_matplotlib

## 데이터 불러오기

- 마스터 테이블

In [118]:
master_df = pd.read_csv('./add/master_last_v5.csv')
master_df.head()

,user_id,total_votes,unique_days,first_vote,last_vote,active_days,cohort_day,retention_day8,gender,school_id,grade,group_id,votes_within_3d,chosen_user_count,period_friend_count,retention_group
0,838466,129,7,2023-05-02,2023-06-02,31,2023-05-02,0.569444,F,369.0,2.0,321.0,77,28,63.0,top25
1,838642,10,2,2023-04-28,2023-04-29,1,2023-04-28,0.560000,F,271.0,2.0,356.0,10,6,6.0,top25
2,840512,141,8,2023-05-02,2023-07-11,70,2023-05-02,0.569444,M,369.0,1.0,930.0,69,45,16.0,top25
3,840685,567,21,2023-05-02,2023-06-06,35,2023-05-02,0.569444,F,369.0,1.0,838.0,259,55,69.0,top25
4,840902,250,14,2023-05-02,2023-05-27,25,2023-05-02,0.569444,F,369.0,2.0,1058.0,73,32,60.0,top25


In [148]:
master_df.shape

(2507, 16)

In [119]:
master_df['school_id'].value_counts()

school_id
352.0     382
5520.0    318
5491.0    271
4426.0    267
4516.0    256
369.0     248
5372.0    220
1478.0    189
271.0     180
1719.0    176
Name: count, dtype: int64

## 파생변수 추가

- 유저의 지역

In [120]:
school_df = pd.read_parquet('./votes/accounts_school.parquet')
school_df.head()

,id,address,student_count,school_type
0,4,충청북도 충주시,239,H
1,5,충청북도 충주시,160,M
2,6,충청북도 충주시,200,H
3,7,충청북도 충주시,114,H
4,8,충청북도 충주시,139,M


In [121]:
# 유저 학교의 주소, 유저 학교의 학생 수, 유저 학교의 타입 정보 추가
adress_questioncategory_df = pd.merge(master_df, school_df, left_on='school_id', right_on='id',  how='left')
# 중복되는 학교 id 컬럼 삭제 
adress_questioncategory_df = adress_questioncategory_df.drop(columns=['id'])
adress_questioncategory_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2507 entries, 0 to 2506
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   user_id              2507 non-null   int64  
 1   total_votes          2507 non-null   int64  
 2   unique_days          2507 non-null   int64  
 3   first_vote           2507 non-null   object 
 4   last_vote            2507 non-null   object 
 5   active_days          2507 non-null   int64  
 6   cohort_day           2507 non-null   object 
 7   retention_day8       2507 non-null   float64
 8   gender               2507 non-null   object 
 9   school_id            2507 non-null   float64
 10  grade                2507 non-null   float64
 11  group_id             2507 non-null   float64
 12  votes_within_3d      2507 non-null   int64  
 13  chosen_user_count    2507 non-null   int64  
 14  period_friend_count  2507 non-null   float64
 15  retention_group      2507 non-null   o

In [122]:
adress_questioncategory_df.head()

,user_id,total_votes,unique_days,first_vote,last_vote,active_days,cohort_day,retention_day8,gender,school_id,grade,group_id,votes_within_3d,chosen_user_count,period_friend_count,retention_group,address,student_count,school_type
0,838466,129,7,2023-05-02,2023-06-02,31,2023-05-02,0.569444,F,369.0,2.0,321.0,77,28,63.0,top25,충청남도 아산시,578,H
1,838642,10,2,2023-04-28,2023-04-29,1,2023-04-28,0.560000,F,271.0,2.0,356.0,10,6,6.0,top25,충청남도 천안시 서북구,491,H
2,840512,141,8,2023-05-02,2023-07-11,70,2023-05-02,0.569444,M,369.0,1.0,930.0,69,45,16.0,top25,충청남도 아산시,578,H
3,840685,567,21,2023-05-02,2023-06-06,35,2023-05-02,0.569444,F,369.0,1.0,838.0,259,55,69.0,top25,충청남도 아산시,578,H
4,840902,250,14,2023-05-02,2023-05-27,25,2023-05-02,0.569444,F,369.0,2.0,1058.0,73,32,60.0,top25,충청남도 아산시,578,H


---

- 첫 질문 카테고리

In [123]:
# 질문 유형 정보 테이블
question_category_df = pd.read_parquet('./polls_question_add_one_category.parquet')
question_category_df.head()

,id,question_text,created_at,category
0,99,가장 신비한 매력이 있는 사람은?,2023-03-31 15:22:53,[상상]
1,100,"""이 사람으로 한 번 살아보고 싶다"" 하는 사람은?",2023-03-31 15:22:53,[상상]
2,101,미래의 틱톡커는?,2023-03-31 15:22:54,[미래]
3,102,여기서 제일 특이한 친구는?,2023-03-31 15:22:54,[인간관계]
4,103,가장 지켜주고 싶은 사람은?,2023-03-31 15:22:55,[인간관계]


In [124]:
# 질문 내용
question_df = pd.read_parquet('./votes/polls_question.parquet')
question_df.head()

,id,question_text,created_at
0,99,가장 신비한 매력이 있는 사람은?,2023-03-31 15:22:53
1,100,"""이 사람으로 한 번 살아보고 싶다"" 하는 사람은?",2023-03-31 15:22:53
2,101,미래의 틱톡커는?,2023-03-31 15:22:54
3,102,여기서 제일 특이한 친구는?,2023-03-31 15:22:54
4,103,가장 지켜주고 싶은 사람은?,2023-03-31 15:22:55


In [125]:
# 질문 정보
questionpiece_df = pd.read_parquet('./votes/polls_questionpiece.parquet')
questionpiece_df.head()

,id,is_voted,created_at,question_id,is_skipped
0,998458,1,2023-04-28 12:27:22,252,0
1,998459,1,2023-04-28 12:27:22,244,0
2,998460,1,2023-04-28 12:27:22,183,0
3,998461,1,2023-04-28 12:27:22,101,0
4,998462,1,2023-04-28 12:27:22,209,0


In [126]:
# 투표 기록 테이블
userquestionrecord_df = pd.read_parquet('votes/accounts_userquestionrecord.parquet') 
userquestionrecord_df.head()

,id,status,created_at,chosen_user_id,question_id,user_id,question_piece_id,has_read,answer_status,answer_updated_at,report_count,opened_times
0,771777,C,2023-04-28 12:27:49,849469,252,849436,998458,0,N,2023-04-28 12:27:49,0,0
1,771800,C,2023-04-28 12:28:02,849446,244,849436,998459,0,N,2023-04-28 12:28:02,0,0
2,771812,C,2023-04-28 12:28:09,849454,183,849436,998460,1,N,2023-04-28 12:28:09,0,0
3,771828,C,2023-04-28 12:28:16,847375,101,849436,998461,0,N,2023-04-28 12:28:16,0,0
4,771851,C,2023-04-28 12:28:26,849477,209,849436,998462,1,N,2023-04-28 12:28:26,0,0


In [127]:
# 유저별 첫번째 질문 id
firstquestion_df = userquestionrecord_df.groupby(['user_id'])['question_id'].first().reset_index()
# 컬럼명 바꾸기
firstquestion_df = firstquestion_df.rename(columns={'question_id': 'firstquestion_id'})
firstquestion_df.head()

,user_id,firstquestion_id
0,838023,121
1,838466,298
2,838642,161
3,839357,179
4,840293,464


In [128]:
# 질문 카테고리와 병합
user_firstquestion_df = pd.merge(firstquestion_df, question_category_df, left_on='firstquestion_id', right_on='id', how='left')
# 필요없는 컬럼 버리기
user_firstquestion_df = user_firstquestion_df.drop(columns=['id','created_at'])
user_firstquestion_df.head()

,user_id,firstquestion_id,question_text,category
0,838023,121,가장 어른스러운 사람은?,[성격]
1,838466,298,옷이 제일 많을 거 같은 사람은?,[스타일]
2,838642,161,화목한 가정을 꾸릴거 같은 사람은?,[인간관계]
3,839357,179,책을 제일 많이 읽은 것 같은 사람은?,[지식]
4,840293,464,뭔가 제대로 한 번 보여줄 것 같은 사람은?,[미래]


In [129]:
# 마스터테이블과 합치기
adress_questioncategory_df = pd.merge(adress_questioncategory_df, user_firstquestion_df, on='user_id', how='left')
adress_questioncategory_df.head()

,user_id,total_votes,unique_days,first_vote,last_vote,active_days,cohort_day,retention_day8,gender,school_id,...,votes_within_3d,chosen_user_count,period_friend_count,retention_group,address,student_count,school_type,firstquestion_id,question_text,category
0,838466,129,7,2023-05-02,2023-06-02,31,2023-05-02,0.569444,F,369.0,...,77,28,63.0,top25,충청남도 아산시,578,H,298,옷이 제일 많을 거 같은 사람은?,[스타일]
1,838642,10,2,2023-04-28,2023-04-29,1,2023-04-28,0.560000,F,271.0,...,10,6,6.0,top25,충청남도 천안시 서북구,491,H,161,화목한 가정을 꾸릴거 같은 사람은?,[인간관계]
2,840512,141,8,2023-05-02,2023-07-11,70,2023-05-02,0.569444,M,369.0,...,69,45,16.0,top25,충청남도 아산시,578,H,219,마술이 눈속임이 아니라 마법이라고 생각할 것 같은 사람은?,[상상]
3,840685,567,21,2023-05-02,2023-06-06,35,2023-05-02,0.569444,F,369.0,...,259,55,69.0,top25,충청남도 아산시,578,H,120,본인 방이 제일 깨끗할거 같은 사람은?,[스타일]
4,840902,250,14,2023-05-02,2023-05-27,25,2023-05-02,0.569444,F,369.0,...,73,32,60.0,top25,충청남도 아산시,578,H,332,선물 고르는 센스가 가장 좋을 것 같은 사람은?,[스타일]


---

In [130]:
# 새로 만든 컬럼만 내보내기

# 컬럼명 재정의
adress_questioncategory_df = adress_questioncategory_df.rename(columns={'question_text':'first_question_text', 'category':'first_question_category'})
user_adress_questioncategory_df = adress_questioncategory_df[['user_id', 'address', 'student_count', 'school_type', 'firstquestion_id', 'first_question_text', 'first_question_category']]
user_adress_questioncategory_df.head()

,user_id,address,student_count,school_type,firstquestion_id,first_question_text,first_question_category
0,838466,충청남도 아산시,578,H,298,옷이 제일 많을 거 같은 사람은?,[스타일]
1,838642,충청남도 천안시 서북구,491,H,161,화목한 가정을 꾸릴거 같은 사람은?,[인간관계]
2,840512,충청남도 아산시,578,H,219,마술이 눈속임이 아니라 마법이라고 생각할 것 같은 사람은?,[상상]
3,840685,충청남도 아산시,578,H,120,본인 방이 제일 깨끗할거 같은 사람은?,[스타일]
4,840902,충청남도 아산시,578,H,332,선물 고르는 센스가 가장 좋을 것 같은 사람은?,[스타일]


In [103]:
# 내보내기
user_adress_questioncategory_df.to_csv('user_adress_questioncategory_df.csv')

---

In [131]:
# 합치기
integrated_master_df = pd.merge(master_df, user_adress_questioncategory_df, on='user_id', how='left')
integrated_master_df.head()

,user_id,total_votes,unique_days,first_vote,last_vote,active_days,cohort_day,retention_day8,gender,school_id,...,votes_within_3d,chosen_user_count,period_friend_count,retention_group,address,student_count,school_type,firstquestion_id,first_question_text,first_question_category
0,838466,129,7,2023-05-02,2023-06-02,31,2023-05-02,0.569444,F,369.0,...,77,28,63.0,top25,충청남도 아산시,578,H,298,옷이 제일 많을 거 같은 사람은?,[스타일]
1,838642,10,2,2023-04-28,2023-04-29,1,2023-04-28,0.560000,F,271.0,...,10,6,6.0,top25,충청남도 천안시 서북구,491,H,161,화목한 가정을 꾸릴거 같은 사람은?,[인간관계]
2,840512,141,8,2023-05-02,2023-07-11,70,2023-05-02,0.569444,M,369.0,...,69,45,16.0,top25,충청남도 아산시,578,H,219,마술이 눈속임이 아니라 마법이라고 생각할 것 같은 사람은?,[상상]
3,840685,567,21,2023-05-02,2023-06-06,35,2023-05-02,0.569444,F,369.0,...,259,55,69.0,top25,충청남도 아산시,578,H,120,본인 방이 제일 깨끗할거 같은 사람은?,[스타일]
4,840902,250,14,2023-05-02,2023-05-27,25,2023-05-02,0.569444,F,369.0,...,73,32,60.0,top25,충청남도 아산시,578,H,332,선물 고르는 센스가 가장 좋을 것 같은 사람은?,[스타일]


In [132]:
integrated_master_df.columns

Index(['user_id', 'total_votes', 'unique_days', 'first_vote', 'last_vote',
       'active_days', 'cohort_day', 'retention_day8', 'gender', 'school_id',
       'grade', 'group_id', 'votes_within_3d', 'chosen_user_count',
       'period_friend_count', 'retention_group', 'address', 'student_count',
       'school_type', 'firstquestion_id', 'first_question_text',
       'first_question_category'],
      dtype='object')

In [133]:
# 내보내기
integrated_master_df.to_csv('master_last_v5.csv')

---

- 투표 받은 유저 수

In [134]:
integrated_df = pd.read_csv('./master_last_v5.csv')
integrated_df.head()

,Unnamed: 0,user_id,total_votes,unique_days,first_vote,last_vote,active_days,cohort_day,retention_day8,gender,...,votes_within_3d,chosen_user_count,period_friend_count,retention_group,address,student_count,school_type,firstquestion_id,first_question_text,first_question_category
0,0,838466,129,7,2023-05-02,2023-06-02,31,2023-05-02,0.569444,F,...,77,28,63.0,top25,충청남도 아산시,578,H,298,옷이 제일 많을 거 같은 사람은?,['스타일']
1,1,838642,10,2,2023-04-28,2023-04-29,1,2023-04-28,0.560000,F,...,10,6,6.0,top25,충청남도 천안시 서북구,491,H,161,화목한 가정을 꾸릴거 같은 사람은?,['인간관계']
2,2,840512,141,8,2023-05-02,2023-07-11,70,2023-05-02,0.569444,M,...,69,45,16.0,top25,충청남도 아산시,578,H,219,마술이 눈속임이 아니라 마법이라고 생각할 것 같은 사람은?,['상상']
3,3,840685,567,21,2023-05-02,2023-06-06,35,2023-05-02,0.569444,F,...,259,55,69.0,top25,충청남도 아산시,578,H,120,본인 방이 제일 깨끗할거 같은 사람은?,['스타일']
4,4,840902,250,14,2023-05-02,2023-05-27,25,2023-05-02,0.569444,F,...,73,32,60.0,top25,충청남도 아산시,578,H,332,선물 고르는 센스가 가장 좋을 것 같은 사람은?,['스타일']


In [135]:
integrated_df.columns

Index(['Unnamed: 0', 'user_id', 'total_votes', 'unique_days', 'first_vote',
       'last_vote', 'active_days', 'cohort_day', 'retention_day8', 'gender',
       'school_id', 'grade', 'group_id', 'votes_within_3d',
       'chosen_user_count', 'period_friend_count', 'retention_group',
       'address', 'student_count', 'school_type', 'firstquestion_id',
       'first_question_text', 'first_question_category'],
      dtype='object')

In [136]:
# 심층 분석에 해당하는 유저 리스트
user_list = list(integrated_df['user_id'].unique())

In [137]:
# 투표에 선택된 기록 보기 위해서 투표 기록 가져옴
userquestionrecord_df.head()

,id,status,created_at,chosen_user_id,question_id,user_id,question_piece_id,has_read,answer_status,answer_updated_at,report_count,opened_times
0,771777,C,2023-04-28 12:27:49,849469,252,849436,998458,0,N,2023-04-28 12:27:49,0,0
1,771800,C,2023-04-28 12:28:02,849446,244,849436,998459,0,N,2023-04-28 12:28:02,0,0
2,771812,C,2023-04-28 12:28:09,849454,183,849436,998460,1,N,2023-04-28 12:28:09,0,0
3,771828,C,2023-04-28 12:28:16,847375,101,849436,998461,0,N,2023-04-28 12:28:16,0,0
4,771851,C,2023-04-28 12:28:26,849477,209,849436,998462,1,N,2023-04-28 12:28:26,0,0


In [138]:
# 기간 자르기
userquestionrecord_chosen_df = userquestionrecord_df[(userquestionrecord_df['created_at']>='2023-04-28')& (userquestionrecord_df['created_at']<'2023-08-01')]
# 심층 분석 유저 데이터만 남기기
userquestionrecord_chosen_df = userquestionrecord_chosen_df[userquestionrecord_chosen_df['chosen_user_id'].isin(user_list)]

# 유저 별 투표 받은 수 확인하기
userquestionrecord_chosen_df = userquestionrecord_chosen_df.groupby(['chosen_user_id']).size().reset_index(name='chosen_count')
userquestionrecord_chosen_df.head()

,chosen_user_id,chosen_count
0,838466,342
1,838642,40
2,840512,309
3,840685,256
4,840902,201


In [139]:
# 마스터 테이블과 합치기
integrated_v5_df = pd.merge(integrated_df, userquestionrecord_chosen_df, left_on='user_id', right_on='chosen_user_id', how='left')
integrated_v5_df.head()

,Unnamed: 0,user_id,total_votes,unique_days,first_vote,last_vote,active_days,cohort_day,retention_day8,gender,...,period_friend_count,retention_group,address,student_count,school_type,firstquestion_id,first_question_text,first_question_category,chosen_user_id,chosen_count
0,0,838466,129,7,2023-05-02,2023-06-02,31,2023-05-02,0.569444,F,...,63.0,top25,충청남도 아산시,578,H,298,옷이 제일 많을 거 같은 사람은?,['스타일'],838466.0,342.0
1,1,838642,10,2,2023-04-28,2023-04-29,1,2023-04-28,0.560000,F,...,6.0,top25,충청남도 천안시 서북구,491,H,161,화목한 가정을 꾸릴거 같은 사람은?,['인간관계'],838642.0,40.0
2,2,840512,141,8,2023-05-02,2023-07-11,70,2023-05-02,0.569444,M,...,16.0,top25,충청남도 아산시,578,H,219,마술이 눈속임이 아니라 마법이라고 생각할 것 같은 사람은?,['상상'],840512.0,309.0
3,3,840685,567,21,2023-05-02,2023-06-06,35,2023-05-02,0.569444,F,...,69.0,top25,충청남도 아산시,578,H,120,본인 방이 제일 깨끗할거 같은 사람은?,['스타일'],840685.0,256.0
4,4,840902,250,14,2023-05-02,2023-05-27,25,2023-05-02,0.569444,F,...,60.0,top25,충청남도 아산시,578,H,332,선물 고르는 센스가 가장 좋을 것 같은 사람은?,['스타일'],840902.0,201.0


In [140]:
# 필요없는 컬럼 삭제
integrated_v5_df = integrated_v5_df.drop(columns=['chosen_user_id'])

In [141]:
integrated_v5_df.columns

Index(['Unnamed: 0', 'user_id', 'total_votes', 'unique_days', 'first_vote',
       'last_vote', 'active_days', 'cohort_day', 'retention_day8', 'gender',
       'school_id', 'grade', 'group_id', 'votes_within_3d',
       'chosen_user_count', 'period_friend_count', 'retention_group',
       'address', 'student_count', 'school_type', 'firstquestion_id',
       'first_question_text', 'first_question_category', 'chosen_count'],
      dtype='object')

In [142]:
# 내보내기
integrated_v5_df.to_csv('master_last_v5.csv')

In [144]:
integrated_df = pd.read_csv('./master_last_v5.csv')
integrated_df.head()

,Unnamed: 0.1,Unnamed: 0,user_id,total_votes,unique_days,first_vote,last_vote,active_days,cohort_day,retention_day8,...,chosen_user_count,period_friend_count,retention_group,address,student_count,school_type,firstquestion_id,first_question_text,first_question_category,chosen_count
0,0,0,838466,129,7,2023-05-02,2023-06-02,31,2023-05-02,0.569444,...,28,63.0,top25,충청남도 아산시,578,H,298,옷이 제일 많을 거 같은 사람은?,['스타일'],342.0
1,1,1,838642,10,2,2023-04-28,2023-04-29,1,2023-04-28,0.560000,...,6,6.0,top25,충청남도 천안시 서북구,491,H,161,화목한 가정을 꾸릴거 같은 사람은?,['인간관계'],40.0
2,2,2,840512,141,8,2023-05-02,2023-07-11,70,2023-05-02,0.569444,...,45,16.0,top25,충청남도 아산시,578,H,219,마술이 눈속임이 아니라 마법이라고 생각할 것 같은 사람은?,['상상'],309.0
3,3,3,840685,567,21,2023-05-02,2023-06-06,35,2023-05-02,0.569444,...,55,69.0,top25,충청남도 아산시,578,H,120,본인 방이 제일 깨끗할거 같은 사람은?,['스타일'],256.0
4,4,4,840902,250,14,2023-05-02,2023-05-27,25,2023-05-02,0.569444,...,32,60.0,top25,충청남도 아산시,578,H,332,선물 고르는 센스가 가장 좋을 것 같은 사람은?,['스타일'],201.0


---

## chosen_user_count 삭제하기 (v4, v5 모두)

In [164]:
integrated_v4_df = pd.read_csv('./master_last_v4.csv')
integrated_v4_df

,Unnamed: 0,user_id,total_votes,unique_days,first_vote,last_vote,active_days,cohort_day,retention_day8,gender,...,votes_within_3d,period_friend_count,retention_group,address,student_count,school_type,firstquestion_id,first_question_text,first_question_category,chosen_count
0,0,838023,22,5,2023-04-29,2023-05-04,5,2023-04-29,0.515152,F,...,7,0.0,mid,충청남도 천안시 서북구,491,H,121,가장 어른스러운 사람은?,['성격'],262.0
1,1,838466,129,7,2023-05-02,2023-06-02,31,2023-05-02,0.569444,F,...,77,63.0,top25,충청남도 아산시,578,H,298,옷이 제일 많을 거 같은 사람은?,['스타일'],342.0
2,2,838642,10,2,2023-04-28,2023-04-29,1,2023-04-28,0.560000,F,...,10,6.0,top25,충청남도 천안시 서북구,491,H,161,화목한 가정을 꾸릴거 같은 사람은?,['인간관계'],40.0
3,3,839357,212,9,2023-04-29,2023-05-26,27,2023-04-29,0.515152,M,...,114,53.0,mid,충청남도 천안시 서북구,491,H,179,책을 제일 많이 읽은 것 같은 사람은?,['지식'],173.0
4,4,840293,170,7,2023-05-03,2023-05-15,12,2023-05-03,0.422977,F,...,130,20.0,mid,충청남도 아산시,578,H,464,뭔가 제대로 한 번 보여줄 것 같은 사람은?,['미래'],241.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4698,4698,1510561,475,19,2023-05-25,2023-06-13,19,2023-05-25,0.451613,M,...,68,32.0,mid,경상남도 거제시,483,H,504,얘랑 있으면 시간이 너무 빨리가는것 같아,['일상'],37.0
4699,4699,1511957,103,11,2023-05-25,2023-06-12,18,2023-05-25,0.451613,F,...,23,15.0,mid,경기도 광명시,500,M,1576,죽기 전에 마지막으로 대화하고 싶은 사람은?,['인간관계'],146.0
4700,4700,1512422,243,8,2023-05-25,2023-06-01,7,2023-05-25,0.451613,F,...,127,11.0,mid,경상남도 거제시,483,H,604,부모님한테 가장 잘할 것 같은 친구는?,['인간관계'],51.0
4701,4701,1512542,636,12,2023-05-25,2023-06-06,12,2023-05-25,0.451613,F,...,167,28.0,mid,경상남도 거제시,483,H,1532,이 사람의 발냄새 만큼은 용서 된다,['일상'],86.0


In [165]:
integrated_v5_df = pd.read_csv('./master_last_v5.csv')
integrated_v5_df

,Unnamed: 0,user_id,total_votes,unique_days,first_vote,last_vote,active_days,cohort_day,retention_day8,gender,...,votes_within_3d,period_friend_count,retention_group,address,student_count,school_type,firstquestion_id,first_question_text,first_question_category,chosen_count
0,0,838466,129,7,2023-05-02,2023-06-02,31,2023-05-02,0.569444,F,...,77,63.0,top25,충청남도 아산시,578,H,298,옷이 제일 많을 거 같은 사람은?,['스타일'],342.0
1,1,838642,10,2,2023-04-28,2023-04-29,1,2023-04-28,0.560000,F,...,10,6.0,top25,충청남도 천안시 서북구,491,H,161,화목한 가정을 꾸릴거 같은 사람은?,['인간관계'],40.0
2,2,840512,141,8,2023-05-02,2023-07-11,70,2023-05-02,0.569444,M,...,69,16.0,top25,충청남도 아산시,578,H,219,마술이 눈속임이 아니라 마법이라고 생각할 것 같은 사람은?,['상상'],309.0
3,3,840685,567,21,2023-05-02,2023-06-06,35,2023-05-02,0.569444,F,...,259,69.0,top25,충청남도 아산시,578,H,120,본인 방이 제일 깨끗할거 같은 사람은?,['스타일'],256.0
4,4,840902,250,14,2023-05-02,2023-05-27,25,2023-05-02,0.569444,F,...,73,60.0,top25,충청남도 아산시,578,H,332,선물 고르는 센스가 가장 좋을 것 같은 사람은?,['스타일'],201.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2502,2502,1497413,335,13,2023-05-24,2023-07-28,65,2023-05-24,0.294118,M,...,152,35.0,bottom25,경상남도 거제시,483,H,1046,졸업 후 동창회에서 청첩장 돌릴 것 같은 친구는?,['인간관계'],199.0
2503,2503,1497699,15,2,2023-05-24,2023-05-25,1,2023-05-24,0.294118,M,...,15,2.0,bottom25,충청남도 천안시 서북구,491,H,336,제대로 놀 때 부르고 싶은 친구는?,['인간관계'],12.0
2504,2504,1498168,268,15,2023-05-24,2023-06-09,16,2023-05-24,0.294118,M,...,54,23.0,bottom25,경상남도 거제시,483,H,911,답장 속도가 가장 빠른 사람은?,['인간관계'],146.0
2505,2505,1498266,4,1,2023-05-24,2023-05-24,0,2023-05-24,0.294118,F,...,4,28.0,bottom25,울산광역시 울주군,550,H,914,예술감각이 가장 뛰어난 친구는?,['스타일'],34.0


In [149]:
# chosen_user_count 삭제
integrated_v4_df = integrated_v4_df.drop(columns=['chosen_user_count'])
integrated_v5_df = integrated_v5_df.drop(columns=['chosen_user_count'])

print(integrated_v4_df.columns)
print(integrated_v5_df.columns)

Index(['Unnamed: 0.1', 'Unnamed: 0', 'user_id', 'total_votes', 'unique_days',
       'first_vote', 'last_vote', 'active_days', 'cohort_day',
       'retention_day8', 'gender', 'school_id', 'grade', 'group_id',
       'votes_within_3d', 'period_friend_count', 'retention_group', 'address',
       'student_count', 'school_type', 'firstquestion_id',
       'first_question_text', 'first_question_category', 'chosen_count'],
      dtype='object')
Index(['Unnamed: 0.1', 'Unnamed: 0', 'user_id', 'total_votes', 'unique_days',
       'first_vote', 'last_vote', 'active_days', 'cohort_day',
       'retention_day8', 'gender', 'school_id', 'grade', 'group_id',
       'votes_within_3d', 'period_friend_count', 'retention_group', 'address',
       'student_count', 'school_type', 'firstquestion_id',
       'first_question_text', 'first_question_category', 'chosen_count'],
      dtype='object')


In [ ]:
# 내보내기
# integrated_v4_df.to_csv('master_last_v4.csv')
# integrated_v5_df.to_csv('master_last_v5.csv')